# Demo: AI-Powered Medical Diagnosis Support

This notebook demonstrates how a DSL can enforce clinical guidelines on an AI's output, inspired by the **VOLUNTIS Decision Table** example from the presentation (Slide 16).

**The Goal:**
An AI assistant reads a patient's vital signs (systolic and diastolic blood pressure) and suggests a risk category. Our DSL acts as a "safety net" to ensure the AI's suggestion conforms to the established medical rules, preventing potentially dangerous diagnostic errors.

**The Technology:**
* **Language:** Python
* **Rules Engine:** `business-rules` library
* **DSL Format:** JSON

---

In [1]:
# First, we import the library that will run our rules.
from business_rules.engine import run_all

# --- Define the DSL for our Travel Expense Policy ---
# These are the rules that our "LLM" must follow.
# The rules are defined as a list of JSON-like Python dictionaries.

travel_expense_rules = [
    {
        "conditions": {
            "all": [
                {
                    "name": "employee_title",
                    "operator": "not_equal_to",
                    "value": "VP"
                },
                {
                    "name": "claim_flight_class",
                    "operator": "equal_to",
                    "value": "Business"
                }
            ]
        },
        "actions": [
            {
                "name": "add_validation_flag",
                "params": {
                    "message": "Business class travel is restricted to VPs."
                }
            }
        ]
    },
    {
        "conditions": {
            "all": [
                {
                    "name": "claim_meal_cost",
                    "operator": "greater_than",
                    "value": 75
                }
            ]
        },
        "actions": [
            {
                "name": "add_validation_flag",
                "params": {
                    "message": "Meal expense exceeds the $75 daily limit."
                }
            }
        ]
    }
]

# This print statement is just to confirm the cell ran correctly.
print("Travel Expense Rules defined successfully.")

Travel Expense Rules defined successfully.


In [2]:
# We need the 'json' library to print our dictionary in a nice, readable way.
import json

# --- Define the User Query and Simulated LLM Responses ---

# This is the user's request that we are simulating.
user_query = "Please book a flight and hotel for our new intern, John Doe. He needs to fly to the conference in New York. Book him a standard flight and find a nice hotel. His meal budget is standard."

# This is our hardcoded list of potential responses from the LLM.
# We will test each of these against our rules.
llm_responses = {
    "good_response": {
        "scenario_name": "Compliant Claim",
        "employee_title": "Intern",
        "claim_flight_class": "Economy",
        "claim_meal_cost": 65
    },
    "bad_response_flight": {
        "scenario_name": "Flight Policy Violation",
        "employee_title": "Intern",
        "claim_flight_class": "Business", # This violates the "VP" rule
        "claim_meal_cost": 70
    },
    "bad_response_meal": {
        "scenario_name": "Meal Policy Violation",
        "employee_title": "Intern",
        "claim_flight_class": "Economy",
        "claim_meal_cost": 90 # This violates the "$75" rule
    }
}

# --- Display the defined data ---
# As requested, we will now print the data we just created to show what's inside.
print("--- User Query ---")
print(user_query)
print("\n--- Simulated LLM Responses (as a Python Dictionary) ---")
print(json.dumps(llm_responses, indent=2))

--- User Query ---
Please book a flight and hotel for our new intern, John Doe. He needs to fly to the conference in New York. Book him a standard flight and find a nice hotel. His meal budget is standard.

--- Simulated LLM Responses (as a Python Dictionary) ---
{
  "good_response": {
    "scenario_name": "Compliant Claim",
    "employee_title": "Intern",
    "claim_flight_class": "Economy",
    "claim_meal_cost": 65
  },
  "bad_response_flight": {
    "scenario_name": "Flight Policy Violation",
    "employee_title": "Intern",
    "claim_flight_class": "Business",
    "claim_meal_cost": 70
  },
  "bad_response_meal": {
    "scenario_name": "Meal Policy Violation",
    "employee_title": "Intern",
    "claim_flight_class": "Economy",
    "claim_meal_cost": 90
  }
}


In [ ]:
from business_rules.variables import BaseVariables, string_rule_variable, numeric_rule_variable
from business_rules.actions import BaseActions, rule_action

# --- Define the "Variables" ---
# This class tells the engine how to get the data values from our LLM response.
# Each method corresponds to a "name" in our rule conditions.
class TravelExpenseVariables(BaseVariables):

    def __init__(self, llm_response):
        self.response = llm_response

    @string_rule_variable
    def employee_title(self):
        return self.response['employee_title']

    @string_rule_variable
    def claim_flight_class(self):
        return self.response['claim_flight_class']

    @numeric_rule_variable
    def claim_meal_cost(self):
        return self.response['claim_meal_cost']

# --- Define the "Actions" ---
# This class tells the engine what code to run when a rule's conditions are met.
# Each method corresponds to a "name" in our rule actions.
class TravelExpenseActions(BaseActions):

    def __init__(self):
        # We create a list to store any error messages from failed rules.
        self.validation_flags = []

    # THIS IS THE CORRECTED LINE
    @rule_action(params=[{'fieldType': 'text', 'name': 'message'}])
    def add_validation_flag(self, message):
        self.validation_flags.append(message)


# --- Display a confirmation ---
print("Engine 'Variables' and 'Actions' are now defined.")
print("These classes provide the connection between our rules and our data.")

Engine 'Variables' and 'Actions' are now defined.
These classes provide the connection between our rules and our data.


In [6]:
# --- Execute the Engine for Each Simulated Response ---

# We will now loop through each of our simulated LLM responses
# and run the validation engine against them.

for scenario_key, response_data in llm_responses.items():
    
    print(f"--- -------------------------------------- ---")
    print(f"--- Testing Scenario: {response_data['scenario_name']} ---")
    print(f"--- -------------------------------------- ---\n")
    
    # Step 1: Instantiate our Variables and Actions classes for this specific response.
    # The 'llm_response' data is passed into the variables class.
    variables = TravelExpenseVariables(response_data)
    actions = TravelExpenseActions()

    # Step 2: Run the engine!
    # The engine takes our defined rules, variables, and actions.
    run_all(rule_list=travel_expense_rules,
            defined_variables=variables,
            defined_actions=actions)

    # Step 3: Check the results and print a clear report.
    # We check if the 'validation_flags' list in our actions class has any messages.
    if actions.validation_flags:
        print("🔴 VALIDATION FAILED. The LLM's response violates the following rules:")
        for flag in actions.validation_flags:
            print(f"   - {flag}")
    else:
        print("🟢 VALIDATION PASSED. The LLM's response is compliant.")
    
    print("\n")

--- -------------------------------------- ---
--- Testing Scenario: Compliant Claim ---
--- -------------------------------------- ---



AssertionError: Operator not_equal_to does not exist for type StringType